In [40]:
%%capture
!pip install ht CoolProp fluids

In [69]:
%matplotlib widget
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import ht,fluids,os
import CoolProp as cp
from IPython.display import Latex
from matplotlib.lines import Line2D
from scipy.optimize import brentq
MK = Line2D.markers
mk = [mki for mki in MK.keys()][2:]

In [52]:
D = 50e-3
T_e0 = 800
T_a = 300
T_w = 300
T_ef = 400
T_eref = 450


T_m = (T_a + T_eref)/2
Pr = cp.CoolProp.PropsSI('Prandtl','T',T_m+273,'P',101.3e3,'Air')
mu_a = cp.CoolProp.PropsSI('V','T',T_m+273,'P',101.3e3,'Air')
rho_a = cp.CoolProp.PropsSI('D','T',T_m+273,'P',101.3e3,'Air')
k_a = cp.CoolProp.PropsSI('L','T',T_m+273,'P',101.3e3,'Air')
nu_a = mu_a/rho_a
a_a = nu_a / Pr
k_e = 231
rho_e = 2700
C_e = 1030
S_e = np.pi*(D/2)**2/4
V_e = np.pi*(D/2)**3*4/3

a_c = 0.833e-7
epsilon_e = 0.75
g = 9.8
L = 1

sigma_sb = 5.67e-8
Lc = D
beta = 1/(T_m+273)
Ra = beta * (T_eref-T_a)*g*Lc**3/ nu_a / a_a
Gr = Ra/Pr


Nu = ht.conv_free_immersed.Nu_sphere_Churchill(Pr,Gr)

In [57]:
hconv = Nu * k_a / D
display(Latex(f'$h_{{conv}}={hconv:.1f}$W/m$^2$'))
Bi = hconv * D / k_e
display(Latex(f'$Bi={Bi:.3f}$'))



<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [83]:
def theta(t,RC):
    return np.exp(-1/RC*t)

def Temp(t,Ti,T0,Tinf,RC):
    return (Ti-Tinf)*theta(t,RC)+Tinf - (Tinf+(Ti-T0))

In [84]:
T_a

300

In [111]:
RC = rho_e*C_e*V_e/(S_e*hconv)
display(Latex(f'$RC={RC:.1f}1/s$'))
tfinal1 = brentq(Temp,0,50000,args=(T_e0,T_ef,T_a,RC),disp=False)
display(Latex(f'$t_{{f1}}={tfinal1/3600:.1f}h$'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [112]:
1/RC

2.369351123859062e-05

In [113]:
550**3*sigma_sb*epsilon_e

7.075096875

In [109]:
hrad = ((T_eref+273)+(T_w+273))* ((T_eref+273)**2+(T_w+273)**2) *sigma_sb*epsilon_e
Qrad = epsilon_e*sigma_sb*((T_eref+273)**4-(T_w+273)**4)*np.pi*D**2/4
#Qrad2 = hrad*(T_c-T_a)*2*L
Bi_2 = hrad * D / k_e
display(Latex(f'$Bi_2={Bi_2:.3f}$'))
htotal = (1/hrad**-1+1/hconv**-1)
display(Latex(f'$h_{{conv}}={hconv:.1f}$W/m$^2$'))
display(Latex(f'$h_{{total}}={htotal:.1f}$W/m$^2$'))
display(Latex(f'$h_{{rad}}={hrad:.1f}$W/m$^2$'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [110]:
RC2 = rho_e*V_e*C_e/(S_e*hrad)
display(Latex(f'$RC={RC:.1f}1/s$'))
tfinal2 = brentq(Temp,0,100000,args=(T_e0,T_ef,T_a,RC2),disp=False)
display(Latex(f'$t_{{f2}}={tfinal2/3600:.1f}h$'))
    

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [107]:
RC3 = rho_e*V_e*C_e/(S_e*htotal)
display(Latex(f'$RC={RC3:.1f}1/s$'))
tfinal3 = brentq(Temp,0,100000,args=(T_e0,T_ef,T_a,RC3),disp=False)
display(Latex(f'$t_{{f3}}={tfinal3/3600:.1f}h$'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [38]:
from scipy.special import erfc

In [39]:
def theta (x,a,t,hc,k):
    z = x/(4*a*t)**0.5
    z2 = hc/k*(a*t)**0.5
    arg = hc*x/k+(hc/k)**2*a*t
    return erfc(z)-np.exp(arg)*erfc(z+z2)
    

In [96]:
V_e/S_e

0.13333333333333333